In [1]:
import secrets
import hashlib
import base64
import ecdsa
from sys import byteorder


In [ ]:
class k_box:
    def __init__(self, file = 0, string = 0 ):
        self.__priv_key__ = 0
        self.__pub_key__ = 0
        if file != 0:
            with open(file_name,'r') as file:
                key = file.read()
                assert len(key) == 64 , "Bad len "
                file.close()
                

In [ ]:
    def get_secret_key(self):
        if self.__priv_key__ == 0:
            self.__priv_key__ = secrets.randbits(256).to_bytes(32,byteorder=byteorder)
        return self.__priv_key__

In [2]:
def base58_encode(n):
    alphabet = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'
    b58_string = ""
    # Get the number of leading zeros
    leading_zeros = len(n.hex()) - len(n.hex().lstrip('0'))
    # Convert hex to decimal
#     address_int = int(address_hex, 16)
    address_int = int.from_bytes(n,byteorder="big")
    # Append digits to the start of string
    while address_int > 0:
        digit = address_int % 58
        digit_char = alphabet[digit]
        b58_string = digit_char + b58_string
        address_int //= 58
    # Add ‘1’ for each 2 leading zeros
    ones = leading_zeros // 2
    for one in range(ones):
        b58_string = '1' + b58_string
    return b58_string

In [3]:
def get_secret_key():
    return secrets.randbits(256).to_bytes(32,byteorder=byteorder)


In [4]:
def priv_key_to_WIF(file_name):
    with open(file_name,'r') as file:
        key = file.read()
        file.close()
    key = "80" + key
    key_b = bytes.fromhex(key)
    sha_key1 = hashlib.sha256(hashlib.sha256(key_b).digest()).digest()
    key_b = key_b + sha_key1[0:4]
#     b = bytes.fromhex(key)
    return base58_encode(b)



In [5]:
def get_public_key(priv_key:bytes):
    pub_key = ecdsa.SigningKey.from_string(priv_key, curve=ecdsa.SECP256k1).verifying_key
    return (0x04.to_bytes(1,byteorder=byteorder) + pub_key.to_string())



In [6]:
def get_public_key_compresed(pub_key:bytes):
    return (0x03.to_bytes(1,byteorder=byteorder) if pub_key[-1] % 2 else 0x02.to_bytes(1,byteorder=byteorder)) + pub_key[1:33]

In [7]:

def encrypt_pub_key(pub_key:bytes):
    sha = hashlib.sha256(pub_key).digest()
#     print ("__sha256__  " + sha.hex())
    result = hashlib.new(name='ripemd160', data=sha).digest()    
#     print ("__ripemd160__  " + result.hex())
    return (result)


In [8]:
def get_address(priv_key:bytes):
    pub_key = get_public_key(priv_key)
    print(f"public key___ {pub_key.hex()}")
    e_pub = encrypt_pub_key(pub_key)
#     print(e_pub.hex())
    main_net_key = 0x00.to_bytes(1,byteorder=byteorder) + e_pub
    check_sum = hashlib.sha256(hashlib.sha256(main_net_key).digest()).digest()[:4]
    print(check_sum.hex())
    hex_addr = main_net_key + check_sum
#     print(hex_addr.hex())
    return base58_encode(hex_addr)


In [9]:
def main():
    key = bytes.fromhex("18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725")
    addr = get_address(key)
    print(addr)

if __name__ == "__main__":
    main()

public key___ 0450863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b23522cd470243453a299fa9e77237716103abc11a1df38855ed6f2ee187e9c582ba6
d61967f6
16UwLL9Risc3QfPqBUvKofHmBQ7wMtjvM


In [10]:
# data = b'123'
# a = hashlib.new(name='ripemd160', data=data).digest() 
# print(data)
# print(a.hex())